In [2]:
import pandas as pd
import numpy as np
from scipy.io import loadmat as loadmat
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False

# Suppress warnings`
warnings.filterwarnings("ignore")

In [1]:
import imp

C:\Users\1\AppData\Local\Temp\ipykernel_4020\713651319.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [3]:
import xy

ValueError: invalid literal for int() with base 10: '2005-01-04'

# $因子描述：RSRS$

***线性动量：拟合以下线性模型：high = beta×low + e，所得 beta×R^2***  
反映

## 计算

In [10]:
def cal_rsrs_d(data_low,data_high,data_op,data_cl,days,idx_name):
    _low = xy.rolling_window(data_low,days)
    _high = xy.rolling_window(data_high,days)
    _op = xy.rolling_window(data_op,days)
    _cl = xy.rolling_window(data_cl,days)

    date = [ i[:10] for i in data_low.index][days-1:]

    _rsrs = {}
    count = 0
    
    for l, h, o, c in zip(_low,_high,_op,_cl):
        slp, r = cal(l,h)
        _rsrs[date[count]] = slp

        count += 1

    # (pd.Series(_rsrs).loc['2015-11-04':].shift(1)).rolling(20).mean().plot()
    # pd.Series(xy.Index_rets.loc[idx_name,'2015-11-04':]).cumsum().rolling(20).mean().plot(secondary_y=True)
    
    corr = pd.Series(xy.Index_rets.loc[idx_name,:]).corr(pd.Series(_rsrs).loc[:].shift(1))
    
    print('corr:', corr,'mean:',pd.Series(_rsrs).mean(), 'std:', pd.Series(_rsrs).std())
    
    return pd.Series(_rsrs).loc['2015-11-04':].shift(1)

In [29]:
def Ts_Stg_hold(sig, long_shrd, short_shrd):
    long_point = pd.Series(1,sig[sig>long_shrd].index)
    short_point = pd.Series(-1,sig[sig<short_shrd].index)
    ls = pd.concat([
        long_point,
        short_point
    ])
    
    return pd.Series(ls,index=sig.index).sort_index().ffill()

In [12]:
def cal(x,y):
    slope, intercept = np.polyfit(x, y, 1)
    # annualized_returns = math.pow(math.exp(slope), 250) - 1
    r_squared = 1 - (sum((y - (slope * x + intercept))**2) / ((len(y) - 1) * np.var(y, ddof=1)))
    score = slope * r_squared
    
    return score,r_squared

In [58]:
xy.Index_rets.loc['50',:].dropna()

2004-01-05    0.048899
2004-01-06    0.014004
2004-01-07    0.009898
2004-01-08    0.015059
2004-01-09   -0.018933
                ...   
2023-05-26   -0.001907
2023-05-29   -0.004426
2023-05-30   -0.005157
2023-05-31   -0.016486
2023-06-01    0.002803
Name: 50, Length: 4715, dtype: float64

In [65]:
def loop(idx_name, day_win, Z_size, trd_shrd):
    rsrs50d_10 = cal_rsrs_d(
                        xy.index_low.T[idx_name].dropna(),
                        xy.index_high.T[idx_name].dropna(),
                        xy.op.T[idx_name].dropna(),
                        xy.cl.T[idx_name].dropna(),
                        day_win,
                        idx_name)
    
    rsrs50_10_sm = rsrs50d_10.rolling(Z_size).apply(lambda x: (x[-1]-np.nanmean(x))/np.nanstd(x)).dropna().rolling(1).mean()
    corr = ((rsrs50_10_sm/1)).corr( xy.Index_rets.loc[idx_name,rsrs50_10_sm.index])
    pos = Ts_Stg_hold(((rsrs50_10_sm/1)), trd_shrd, -trd_shrd)

    ((pos * xy.Index_rets.loc[idx_name,rsrs50_10_sm.index])).cumsum().rename('cumret').plot(figsize=(20,8),grid=True,legend=True)
    (xy.Index_rets.loc[idx_name,rsrs50_10_sm.index]).cumsum().plot(secondary_y=True,figsize=(20,8),legend=True)
    
    ret = (pos * xy.Index_rets.loc[idx_name,:])
    sharpe_no_cut = (ret.dropna().sum())/len(ret.dropna())*250 /  ((ret.dropna().std()* 250**0.5))

    res_dct = {
    '标的':idx_name,
    '回归窗宽':day_win,
    'Z值窗宽':Z_size,
    'Z值阈值':trd_shrd,
    '未扣费累计收益':ret.dropna().cumsum()[-1],
    '扣费累计收益':ret.dropna().cumsum()[-1]-pos.diff().abs().sum()*0.0005,
    '最大回撤':-xy.cal_downdraw(ret.dropna()),
    '夏普':sharpe_no_cut,
    '胜率':ret[ret>0].dropna().count() / ret[ret!=0].dropna().count(), 
    '盈亏比':ret[ret>0].sum() / ret[ret<0].sum(), 
    '交易次数':pos.diff().abs().sum(),
    '时序ic':corr
}
    
    
    return res_dct,pd.concat([ret.rename('ret'),pos.rename('position')],axis=1)

In [66]:
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt

idx_names = ['50','300','500','1000']
sample_sizes = [20,25,30,40]
win_sizes = [20,60,120,240]
open_shrds = [1,1.5,2,2.5,3]

In [ ]:
# all_results = {}
all_results_dfs = {}

for idxn, ss, ws, os in tqdm(product(idx_names, sample_sizes, win_sizes, open_shrds)):
    print(f'idx_names:{idxn}_sample_sizes:{ss}_win_sizes:{ws}_open_shrds:{os}')
    res_dct, res_df = loop(idxn, ss, ws, os)


    all_results[f'idx_names:{idxn}_sample_sizes:{ss}_win_sizes:{ws}_open_shrds:{os}'] = res_dct
    res_df.to_csv(f'result/idx_names_{idxn}_sample_sizes_{ss}_win_sizes_{ws}_open_shrds_{os}.csv')
    display(pd.DataFrame(pd.Series(res_dct)).T)
    plt.show()
